In [1]:
# Main imports
import numpy as np
import pandas as pd
import re

# Conversion parameter
kbt_to_kcal = 1/1.62

In [2]:
# Function to evaluate the energy of an PSAM given a DNA site sequence
def compute_energy(df, site):
    L = df.shape[0]
    assert matrix_df.shape[1] == 4, 'matrix has an invalid number of columns'
    assert all(df.columns == list('ACGT')), 'matrix has invalid columns'
    assert len(site)==L, 'site is the wrong length'
    assert set(site) <= set('ACGT'), 'site contains invalid bases'
    
    energy = 0
    for i, c in enumerate(site):
        energy += df.loc[i,c]
    return energy

In [3]:
# Load clonal measurements 
data_df = pd.read_excel('../data/results.xlsx', sheet_name='measurements_summary').set_index('name')
data_df.head()

,location,log_t+,dlog_t+,log_t-,dlog_t-,num_t+,num_t-,outlier,spacing,sequence
name,,,,,,,,,,
61c-oc0,b5E4,-1.691993,0.146085,1.357366,0.088154,6,6,0,0.5,CGCAATGAATCACTCCATTGAGTGTTTTGAGGGTCCCCAGGCTTTA...
61c-ocl,b5A8,-4.955585,0.477209,-2.652523,0.185727,12,6,0,4.5,CGCAATGAATCACTCCATTGAGTGTTTTGAGGGTCCCCAGGCTTTA...
61c-ocl.35L01,b5B2,-5.426847,1.395136,-3.139291,0.053276,15,9,0,4.5,CGCAATGAATCACTCCATTGAGTGTTTTGAGGGTCCCCAGGGTTTA...
61c-ocl.35L02,b5B3,-5.057494,1.232833,-2.840256,0.373761,15,9,0,4.5,CGCAATGAATCACTCCATTGAGTGTTTTGAGGGTCCCCAGGCTTTA...
61c-ocl.35L04,b5B5,-4.600446,0.550925,-4.516905,0.056885,14,6,0,4.5,CGCAATGAATCACTCCATTGAGTGTTTTGAGGGTCCCCACCCTTAA...


In [4]:
# Load spacing results
distance_df = pd.read_excel('../data/results.xlsx', sheet_name='conjoined_resamp').set_index('run')
distance_df.head()

,log_tsat,log_tbg_c60,log_tbg_c61,log_tbg_c62,log_tbg_c63,log_tbg_c64,log_tbg_c65,log_tbg_c66,log_tbg_c71,log_tbg_c72,...,b14A4_log_P,b14A4_weight,b14A5_log_P,b14A5_weight,b14A6_log_P,b14A6_weight,b14A7_log_P,b14A7_weight,b3G7_log_P,b3G7_weight
run,,,,,,,,,,,,,,,,,,,,,
fit,2.722414,-7.061722,-5.531544,-6.633040,-18.662908,-7.025583,-16.005608,-6.207685,-5.607890,-5.890359,...,-5.253013,1,-4.411129,1,-7.409196,1,-9.114505,1,-4.637605,1
samp_00,2.669641,-6.987937,-5.414667,-7.807749,-18.659871,-7.279091,-15.953115,-6.207960,-5.597492,-5.941387,...,NaN,0,NaN,0,-7.342037,3,NaN,0,-4.567086,1
samp_01,2.786586,-6.838136,-5.652310,-5.849840,-18.650824,-6.926201,-15.963883,-6.207557,-5.739206,-5.807886,...,-5.292839,1,-4.457553,1,-7.431922,1,-9.100459,2,NaN,0
samp_02,2.734866,-6.946145,-5.581868,-5.920328,-18.642575,-6.979396,-15.955437,-6.207287,-5.520496,-5.844688,...,-5.256471,2,-4.414873,2,-7.428245,1,NaN,0,-4.640827,1
samp_03,2.689187,-7.441049,-5.270647,-6.297209,-18.640144,-6.837193,-15.958564,-6.207817,-5.738832,-5.843323,...,NaN,0,-4.402817,4,-7.470833,1,NaN,0,-4.631861,1


In [5]:
# Index tmp_df by location
tmp_df = data_df.set_index('location')
tmp_df['name'] = data_df.index

# Create a pattern for RNAP site sequences. 
fixed_left='C'
var_35 = 'AGGCTTTACACCTG'
fixed_middle = 'TTGCCTCCGG'
var_10 = 'CTCGTATGTTGTGT'
fixed_right='GG'
regex = '.*(%s[ACGT]{%d}%s[ACGT]{%d}%s).*'%(fixed_left, len(var_35), fixed_middle, len(var_10), fixed_right)
print('RNAP bindign site pattern: %s'%regex)

# Add RNAP sites to tmp_df
tmp_df['rnap_site'] = ''
for location in tmp_df.index:
    promoter_seq = str(tmp_df.loc[location,'sequence'])
    m = re.match(regex, promoter_seq)
    if m:
        tmp_df.loc[location,'rnap_site'] = m.group(1)

# Remove all columns that don't have a site
rows_to_keep = [len(seq)>0 for seq in tmp_df['rnap_site']]
tmp_df = tmp_df[rows_to_keep]

# Remove all rows with 'c-' in the name
rows_to_keep = [('c-' not in name) for name in tmp_df['name']]
tmp_df = tmp_df[rows_to_keep]

# Compute distance
tmp_df = tmp_df[['rnap_site','spacing']]
tmp_df.head()

RNAP bindign site pattern: .*(C[ACGT]{14}TTGCCTCCGG[ACGT]{14}GG).*


,rnap_site,spacing
location,,
b3C7,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTGTGTGG,-60.5
b7D8,CAGGCTTTACACCTGTTGCCTCCGGCGCGTATGTTGTGTGG,-60.5
b7D9,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTTTGTGG,-60.5
b7E1,CAGGCTTTACACCTGTTGCCTCCGGCTCGTACGTAGTGTGG,-60.5
b7E2,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTGTGTGG,-60.5


In [6]:
# Extract log_Ps and compute mean and variance
tmp_df['dG_mean'] = np.nan
tmp_df['dG_sem'] = np.nan
for i, loc in enumerate(tmp_df.index):
    col_name = '%s_log_P'%loc
    if col_name in distance_df.columns:
        log_Ps = distance_df[col_name].values
        log_Ps = log_Ps[np.isfinite(log_Ps)]
        N = len(log_Ps)
        if N > 1:
            tmp_df.loc[loc,'dG_mean'] = -kbt_to_kcal*np.mean(log_Ps)
            tmp_df.loc[loc,'dG_sem'] = kbt_to_kcal*np.std(log_Ps)
        
tmp_df.dropna(inplace=True)
tmp_df.head()

,rnap_site,spacing,dG_mean,dG_sem
location,,,,
b3C7,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTGTGTGG,-60.5,2.698659,0.033667
b7D8,CAGGCTTTACACCTGTTGCCTCCGGCGCGTATGTTGTGTGG,-60.5,1.820122,0.033057
b7D9,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTTTGTGG,-60.5,3.393222,0.049117
b7E1,CAGGCTTTACACCTGTTGCCTCCGGCTCGTACGTAGTGTGG,-60.5,5.297989,0.128820
b7E2,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTGTGTGG,-60.5,2.929960,0.042702


In [7]:
# Load RNAP PSAM
matrix_df = pd.read_excel('../data/results.xlsx', sheet_name='rnap_motif')
del matrix_df['position']

# Center each row on the nt in the lac* wt sequence
wt_seq = 'CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTGTGTGG'
L = len(wt_seq)
for i, c in enumerate(wt_seq):
    matrix_df.loc[i,:] -= matrix_df.loc[i,c]
    
matrix_df.head()

,A,C,G,T
0,-0.1020,0.0000,-0.1109,-0.0146
1,0.0000,0.2215,-0.0400,0.2827
2,0.0141,-0.0259,0.0000,-0.0499
3,-0.0216,0.1031,0.0000,0.0901
4,0.4600,0.0000,-0.0463,0.9658


In [8]:
tmp_df['ddG_matrix'] = [compute_energy(matrix_df, site) for site in tmp_df['rnap_site']]
tmp_df.to_csv('rnap_summary.csv', sep='\t')
tmp_df.head()

,rnap_site,spacing,dG_mean,dG_sem,ddG_matrix
location,,,,,
b3C7,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTGTGTGG,-60.5,2.698659,0.033667,0.0000
b7D8,CAGGCTTTACACCTGTTGCCTCCGGCGCGTATGTTGTGTGG,-60.5,1.820122,0.033057,-0.3250
b7D9,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTTTGTGG,-60.5,3.393222,0.049117,0.1957
b7E1,CAGGCTTTACACCTGTTGCCTCCGGCTCGTACGTAGTGTGG,-60.5,5.297989,0.128820,1.5180
b7E2,CAGGCTTTACACCTGTTGCCTCCGGCTCGTATGTTGTGTGG,-60.5,2.929960,0.042702,0.0000
